<center>
    <h2>Online learning platform database - MongoDB</h2>
    <h3>Performing the queries and storing the queries execution time</h3>
</center>

<h3>Python - MongoDB interaction</h3>

Prior to performing the queries, we import the required modules (the <i>PyMongo</i> driver and the <i>time</i> and <i>csv</i> modules) and establish a connection to the MongoDB instance running in Docker. It is also convenient to store the database and the collections of interest into Python variables.

In [3]:
# import modules
import pymongo            # MongoDB driver
import time               # time-related functions to register query execution times
import csv                # read and write csv files

# create client object
client = pymongo.MongoClient(host = 'localhost', port = 27017, username = 'root', password = 'root')

# assign database to Python variable (db)
db = client.get_database('dbB_MONGODB_test')

# assign collections to Python variables
smallColl = db.smallDB
mediumColl = db.mediumDB
largeColl = db.largeDB
humongousColl = db.humongousDB

# create cursor object
#cursor = conn.cursor()

<h4>
Measuring and displaying the query execution time
</h4><br>
To display the query execution time we can use the Python <a href = 'https://docs.python.org/3/library/time.html'><i>time</i></a> module and its <code>time</code> function. The function returns the system time at a floating point precision, so the query execution time can be measured as a large number of fractions of a second. It is sufficient to assign the time before the query execution to a variable and the time after the query execution to another variable. The difference between the two variables will measure the query execution. Obviously, the time for the Python API to connect to the MongoDB server and the time to return to the Python API after the query execution will be summed up to the query execution time at the DBMS level.

In [60]:
import time
start = time.time()
cursor1 = smallColl.find({'courseID' : 192}, {'_id' : 0, 'studentID' : 1, 'firstName' : 1, 'lastName' : 1})
end = time.time()
print((end - start) * 1000)

0.1888275146484375


<h3>Query the collections</h3>

I create a dictionary of lists for each of the four collections. In these dictionaries the keys are the query names and the values are the 31 query execution times: in fact I attach the value of the query execution time of the most recent query to the list. Since query execution times are required in milliseconds, prior to attaching them, I multiply them by 1000 and round them to the fifth decimal precision.
The above summarized actions (for each of the four queries on each of the four collections) are performed by following a standard succession of steps. Each step is encapsulated within a notebook cell (so each query is performed 31 times by using three notebook cells), as follows:
 - step 1: define the query, perform it for the first time, contextually create timestamps prior and after query execution, print query result;
 - step 2: compute execution time of the first query execution and store it within the corresponding dictionary list;
 - step 3: [thirty times] perform query execution while creating prior and following timestamps, compute execution time and store it within the corresponding dictionary list. With Mongo I don't need to reset the cursor to allow repeating the query.

For each dataset, after having performed the four queries, I will finally compute the mean of the query executions from step 3. Together with the first query execution, this mean value will be stored into a new dictionary, specific to a dataset. The four new dictionaries will then be saved as a csv file for constructing histograms.

In [15]:
smallDict = {'query1' : list(), 'query2' : list(), 'query3' : list(), 'query4' : list()}
mediumDict = {'query1' : list(), 'query2' : list(), 'query3' : list(), 'query4' : list()}
largeDict = {'query1' : list(), 'query2' : list(), 'query3' : list(), 'query4' : list()}
humongousDict = {'query1' : list(), 'query2' : list(), 'query3' : list(), 'query4' : list()}

In [10]:
# mean function
def mean(aList):
    n = len(aList)
    sum = 0
    for value in aList:
        sum += value
    return sum / n

<h3>Collection with 250k documents</h3><br>
I start with the smallest collection.

<h4>Query 1</h4>

In [16]:
# step 1
small_mongo1 = [{'$match' : {'courseID' : 192}}, {'$group' : {'_id' : {'name' : '$firstName', 'surname' : '$lastName'}}}, {'$project' : {'firstName' : 1, 'lastName' : 1}}]

before = time.time()
small_query1 = smallColl.aggregate(small_mongo1)
after = time.time()

for result in small_query1:
    print(result['_id']['name'], result['_id']['surname'])

Émile Nicolas
Cathrine Lie
Nath Nicolas
Ledün Soylu
Ingeborg Amundsen
Arthur Laroche
Narciso Ferrán
Casandra Arenas
Custodia Hidalgo
Sarah Lara
Patrícia Leite
Vigilija Gaižauskas
Ana Narušis


In [17]:
# step 2
msec_duration = (after - before) * 1000
smallDict['query1'].append(round(msec_duration, 5))

In [18]:
# step 3
for i in range(0, 30):
    before = time.time()
    smallColl.aggregate(small_mongo1)
    after = time.time()
    msec_duration = (after - before) * 1000
    smallDict['query1'].append(round(msec_duration, 5))

In [19]:
smallDict['query1']

[192.01708,
 148.67926,
 127.74396,
 132.73692,
 125.6938,
 130.42593,
 132.64608,
 130.61404,
 134.8269,
 128.96514,
 171.11492,
 144.75203,
 123.18492,
 125.386,
 126.82486,
 127.62499,
 168.33091,
 194.70096,
 128.42607,
 138.58414,
 138.27682,
 148.01908,
 147.14003,
 129.25005,
 123.20209,
 136.40308,
 159.40189,
 141.75892,
 125.15616,
 142.05384,
 191.38885]

<h4>Query 2</h4>

In [30]:
# step 1
small_mongo2 = [{'$match' : {'discipline' : 'statistics', 'courseYear' : 2022}}, {'$group' : {'_id' : {'ID' : '$courseID', 'course' : '$courseName'}}}, {'$project' : {'_id.course' : 1}}]

before = time.time()
small_query2 = smallColl.aggregate(small_mongo2)
after = time.time()

for result in small_query2:
    print(result['_id']['course'])

Foundations: Data, Data, Everywhere
Basic Statistics
Econometrics: Methods and Applications
Exploratory Data Analysis
Bayesian Statistics: From Concept to Data Analysis
Understanding Clinical Research: Behind the Statistics
Introduction to Statistics
Python and Statistics for Financial Analysis
Introduction to Probability and Data with R


In [31]:
# step 2
msec_duration = (after - before) * 1000
smallDict['query2'].append(round(msec_duration, 5))

In [32]:
# step 3
for i in range(0, 30):
    before = time.time()
    smallColl.aggregate(small_mongo2)
    after = time.time()
    msec_duration = (after - before) * 1000
    smallDict['query2'].append(round(msec_duration, 5))

In [33]:
smallDict['query2']

[312.48307,
 260.34093,
 226.48501,
 170.45212,
 166.99171,
 195.08314,
 183.59113,
 181.37288,
 192.99603,
 166.76307,
 170.74394,
 170.82405,
 165.51208,
 178.72095,
 165.86804,
 165.38715,
 181.74601,
 202.47698,
 215.80982,
 180.78423,
 180.35769,
 171.45491,
 268.52679,
 318.87412,
 227.42605,
 235.55899,
 178.04503,
 184.28302,
 186.02514,
 163.92303,
 171.63515]

<h4>Query 3</h4>

In [50]:
# step 1
small_mongo3 = [{'$match' : {'discipline': 'maths', 'materialType' : 'lecture slides', 'email': {'$regex': '@gmail.com'}}}, {'$group' : {'_id': '_id', 'IDcount' : {'$count': {}}}}]

before = time.time()
small_query3 = smallColl.aggregate(small_mongo3)
after = time.time()

for result in small_query3:
    print(result['IDcount'])

838


In [51]:
# step 2
msec_duration = (after - before) * 1000
smallDict['query3'].append(round(msec_duration, 5))

In [52]:
# step 3
for i in range(0, 30):
    before = time.time()
    smallColl.aggregate(small_mongo3)
    after = time.time()
    msec_duration = (after - before) * 1000
    smallDict['query3'].append(round(msec_duration, 5))

In [53]:
smallDict['query3']

[266.86788,
 171.24104,
 142.519,
 146.34204,
 146.21115,
 146.42215,
 176.85294,
 151.75509,
 154.60992,
 146.61407,
 144.40227,
 150.61879,
 149.56284,
 142.98487,
 144.09494,
 171.33808,
 206.3508,
 181.37169,
 177.28996,
 174.52002,
 215.40403,
 200.69766,
 337.74877,
 689.77189,
 299.50404,
 220.75486,
 204.85997,
 162.45794,
 178.18904,
 163.46216,
 178.35379]

<h4>Query 4</h4>

In [45]:
# step 1
small_mongo4 = [{'$match' : {'discipline': 'psychology', 'country' : {'$regex': 'orea'}, 'dateOfBirth': {'$regex': '^1'}}}, {'$project' : {'_id': 0, 'firstName': 1, 'lastName': 1, 'country': 1}}, {'$group' : {'_id': {'name': '$firstName', 'surname': '$lastName', 'country': '$country'}}}, {'$sort' : {'lastName' : 1}}]

before = time.time()
small_query4 = smallColl.aggregate(small_mongo4)
after = time.time()

for result in small_query4:
    print(result['_id']['name'], result['_id']['surname'], 'from', result['_id']['country'])

Lynda Reynolds from Korea
Cathrine Lie from South Korea
Raghav Sura from North Korea


In [46]:
# step 2
msec_duration = (after - before) * 1000
smallDict['query4'].append(round(msec_duration, 5))

In [47]:
# step 3
for i in range(0, 30):
    before = time.time()
    smallColl.aggregate(small_mongo4)
    after = time.time()
    msec_duration = (after - before) * 1000
    smallDict['query4'].append(round(msec_duration, 5))

In [48]:
smallDict['query4']

[208.28009,
 167.99116,
 155.11084,
 152.21214,
 140.56706,
 143.35012,
 145.02001,
 145.39194,
 157.1939,
 159.77097,
 169.04187,
 164.61635,
 167.665,
 158.17714,
 146.0669,
 208.69708,
 151.72577,
 145.94793,
 145.45798,
 152.36402,
 145.38121,
 145.04123,
 164.43181,
 184.00311,
 152.50707,
 145.23387,
 146.12794,
 150.12193,
 149.09196,
 174.86906,
 146.18683]

In [105]:
smallDataset = {'query1' : list(), 'query2' : list(), 'query3' : list(), 'query4' : list()}
for key in smallDict:
    smallDataset[key].append(smallDict[key][0])
    mean30 = mean(smallDict[key][1 : 31])
    smallDataset[key].append(round(mean30, 5))
smallDataset

{'query1': [192.01708, 140.77709],
 'query2': [312.48307, 194.26864],
 'query3': [266.86788, 195.87686],
 'query4': [208.28009, 155.97881]}

<h3>Collection with 500k documents</h3><br>

<h4>Query 1</h4>

In [54]:
# step 1
medium_mongo1 = [{'$match' : {'courseID' : 192}}, {'$group' : {'_id' : {'name' : '$firstName', 'surname' : '$lastName'}}}, {'$project' : {'firstName' : 1, 'lastName' : 1}}]

before = time.time()
medium_query1 = mediumColl.aggregate(medium_mongo1)
after = time.time()

for result in medium_query1:
    print(result['_id']['name'], result['_id']['surname'])

Cathrine Lie
Karl Christensen
Christl Henschel
Vigilija Gaižauskas
Custodia Hidalgo
Nedas Naujokas
Narciso Ferrán
Sarah Lara
Patrícia Leite
Joris Kavaliauskas
Miguel Real
Arthur Laroche
Yuvaan Dara
Ledün Soylu
Ana Narušis
Émile Nicolas
Ingeborg Amundsen
Casandra Arenas
Nath Nicolas
Débora Vaz


In [55]:
# step 2
msec_duration = (after - before) * 1000
mediumDict['query1'].append(round(msec_duration, 5))

In [56]:
# step 3
for i in range(0, 30):
    before = time.time()
    mediumColl.aggregate(medium_mongo1)
    after = time.time()
    msec_duration = (after - before) * 1000
    mediumDict['query1'].append(round(msec_duration, 5))

In [57]:
mediumDict['query1']

[3829.1719,
 420.63785,
 265.49029,
 250.57483,
 265.53607,
 297.94383,
 270.54691,
 319.01789,
 470.95704,
 316.07819,
 432.94883,
 265.35392,
 319.25297,
 265.11884,
 258.12721,
 261.85799,
 262.7039,
 269.06705,
 249.15504,
 255.62811,
 250.27514,
 261.96003,
 254.76503,
 258.63123,
 258.46791,
 257.08222,
 254.91881,
 262.77208,
 252.70796,
 396.41118,
 288.414]

<h4>Query 2</h4>

In [58]:
# step 1
medium_mongo2 = [{'$match' : {'discipline' : 'statistics', 'courseYear' : 2022}}, {'$group' : {'_id' : {'ID' : '$courseID', 'course' : '$courseName'}}}, {'$project' : {'_id.course' : 1}}]

before = time.time()
medium_query2 = mediumColl.aggregate(medium_mongo2)
after = time.time()

for result in medium_query2:
    print(result['_id']['course'])

Bayesian Statistics: From Concept to Data Analysis
Understanding Clinical Research: Behind the Statistics
Python and Statistics for Financial Analysis
Exploratory Data Analysis
Introduction to Statistics
Foundations: Data, Data, Everywhere
Introduction to Probability and Data with R
Basic Statistics
Econometrics: Methods and Applications


In [59]:
# step 2
msec_duration = (after - before) * 1000
mediumDict['query2'].append(round(msec_duration, 5))

In [60]:
# step 3
for i in range(0, 30):
    before = time.time()
    mediumColl.aggregate(medium_mongo2)
    after = time.time()
    msec_duration = (after - before) * 1000
    mediumDict['query2'].append(round(msec_duration, 5))

In [61]:
mediumDict['query2']

[459.23471,
 360.01587,
 326.99275,
 358.72412,
 344.13981,
 380.72777,
 343.70995,
 371.14406,
 430.05896,
 410.56108,
 436.94115,
 384.81688,
 356.00185,
 367.86413,
 401.45206,
 603.53875,
 421.18502,
 354.54512,
 472.13578,
 608.22606,
 471.38596,
 364.39514,
 378.64518,
 351.02201,
 365.80086,
 361.58204,
 339.02001,
 326.17807,
 332.39698,
 331.21514,
 324.44596]

<h4>Query 3</h4>

In [62]:
# step 1
medium_mongo3 = [{'$match' : {'discipline': 'maths', 'materialType' : 'lecture slides', 'email': {'$regex': '@gmail.com'}}}, {'$group' : {'_id': '_id', 'IDcount' : {'$count': {}}}}]

before = time.time()
medium_query3 = mediumColl.aggregate(medium_mongo3)
after = time.time()

for result in medium_query3:
    print(result['IDcount'])

1698


In [63]:
# step 2
msec_duration = (after - before) * 1000
mediumDict['query3'].append(round(msec_duration, 5))

In [64]:
# step 3
for i in range(0, 30):
    before = time.time()
    mediumColl.aggregate(medium_mongo3)
    after = time.time()
    msec_duration = (after - before) * 1000
    mediumDict['query3'].append(round(msec_duration, 5))

In [65]:
mediumDict['query3']

[444.53502,
 360.33583,
 333.11605,
 324.85604,
 536.99517,
 520.97511,
 312.13999,
 311.81884,
 340.22117,
 416.56184,
 297.86229,
 398.80204,
 309.22985,
 727.44775,
 331.0411,
 687.80971,
 603.94597,
 301.05495,
 532.59301,
 520.66112,
 466.151,
 299.11613,
 399.58715,
 512.92515,
 552.08397,
 656.59118,
 465.6179,
 583.46915,
 324.66912,
 305.11928,
 355.57795]

<h4>Query 4</h4>

In [66]:
# step 1
medium_mongo4 = [{'$match' : {'discipline': 'psychology', 'country' : {'$regex': 'orea'}, 'dateOfBirth': {'$regex': '^1'}}}, {'$project' : {'_id': 0, 'firstName': 1, 'lastName': 1, 'country': 1}}, {'$group' : {'_id': {'name': '$firstName', 'surname': '$lastName', 'country': '$country'}}}, {'$sort' : {'lastName' : 1}}]

before = time.time()
medium_query4 = mediumColl.aggregate(medium_mongo4)
after = time.time()

for result in medium_query4:
    print(result['_id']['name'], result['_id']['surname'], 'from', result['_id']['country'])

Raghav Sura from North Korea
Cathrine Lie from South Korea
Lynda Reynolds from Korea
Miguel Real from República de Corea
Ninthe Horrocks from Noord-Korea


In [67]:
# step 2
msec_duration = (after - before) * 1000
mediumDict['query4'].append(round(msec_duration, 5))

In [68]:
# step 3
for i in range(0, 30):
    before = time.time()
    mediumColl.aggregate(medium_mongo4)
    after = time.time()
    msec_duration = (after - before) * 1000
    mediumDict['query4'].append(round(msec_duration, 5))

In [69]:
mediumDict['query4']

[355.16381,
 306.51426,
 263.66687,
 269.09494,
 264.28795,
 364.64071,
 269.60897,
 268.9209,
 340.1289,
 305.56178,
 282.66072,
 269.91415,
 269.768,
 266.93296,
 278.87487,
 263.83424,
 268.6739,
 291.03613,
 283.92816,
 328.439,
 276.11685,
 263.1371,
 266.34502,
 264.503,
 267.22789,
 267.86923,
 317.54398,
 282.74393,
 297.87302,
 283.55408,
 702.53992]

In [104]:
mediumDataset = {'query1' : list(), 'query2' : list(), 'query3' : list(), 'query4' : list()}
for key in mediumDict:
    mediumDataset[key].append(mediumDict[key][0])
    mean30 = mean(mediumDict[key][1 : 31])
    mediumDataset[key].append(round(mean30, 5))
mediumDataset

{'query1': [3829.1719, 290.41341],
 'query2': [459.23471, 389.29562],
 'query3': [444.53502, 436.27919],
 'query4': [355.16381, 298.19805]}

<h3>Collection with 750k documents</h3><br>

<h4>Query 1</h4>

In [70]:
# step 1
large_mongo1 = [{'$match' : {'courseID' : 192}}, {'$group' : {'_id' : {'name' : '$firstName', 'surname' : '$lastName'}}}, {'$project' : {'firstName' : 1, 'lastName' : 1}}]

before = time.time()
large_query1 = largeColl.aggregate(large_mongo1)
after = time.time()

for result in large_query1:
    print(result['_id']['name'], result['_id']['surname'])

Cathrine Lie
Karl Christensen
Christl Henschel
Vigilija Gaižauskas
Custodia Hidalgo
Nedas Naujokas
Patrícia Leite
Narciso Ferrán
Sarah Lara
Joris Kavaliauskas
Miguel Real
Arthur Laroche
Yuvaan Dara
Ledün Soylu
Urvi Dani
Özkutlu Gül
Dorita Abella
Liliana Flaiano
Ana Narušis
Émile Nicolas
Ingeborg Amundsen
Collin Heerkens
Casandra Arenas
Brian Thompson
Nath Nicolas
Débora Vaz


In [71]:
# step 2
msec_duration = (after - before) * 1000
largeDict['query1'].append(round(msec_duration, 5))

In [72]:
# step 3
for i in range(0, 30):
    before = time.time()
    largeColl.aggregate(large_mongo1)
    after = time.time()
    msec_duration = (after - before) * 1000
    largeDict['query1'].append(round(msec_duration, 5))

In [73]:
largeDict['query1']

[9782.52077,
 552.92368,
 607.67078,
 502.83074,
 430.40109,
 515.11002,
 431.52523,
 555.60708,
 436.72991,
 511.12294,
 677.073,
 519.36412,
 417.85812,
 490.18788,
 1200.42419,
 766.65282,
 860.10695,
 729.66123,
 560.61482,
 671.49711,
 801.42093,
 497.12777,
 540.12275,
 690.98496,
 680.54008,
 494.46511,
 808.8131,
 1833.60386,
 401.44205,
 404.32715,
 421.78988]

<h4>Query 2</h4>

In [74]:
# step 1
large_mongo2 = [{'$match' : {'discipline' : 'statistics', 'courseYear' : 2022}}, {'$group' : {'_id' : {'ID' : '$courseID', 'course' : '$courseName'}}}, {'$project' : {'_id.course' : 1}}]

before = time.time()
large_query2 = largeColl.aggregate(large_mongo2)
after = time.time()

for result in large_query2:
    print(result['_id']['course'])

Bayesian Statistics: From Concept to Data Analysis
Understanding Clinical Research: Behind the Statistics
Python and Statistics for Financial Analysis
Exploratory Data Analysis
Introduction to Statistics
Foundations: Data, Data, Everywhere
Introduction to Probability and Data with R
Basic Statistics
Econometrics: Methods and Applications


In [75]:
# step 2
msec_duration = (after - before) * 1000
largeDict['query2'].append(round(msec_duration, 5))

In [76]:
# step 3
for i in range(0, 30):
    before = time.time()
    largeColl.aggregate(large_mongo2)
    after = time.time()
    msec_duration = (after - before) * 1000
    largeDict['query2'].append(round(msec_duration, 5))

In [77]:
largeDict['query2']

[653.68295,
 540.80081,
 543.18285,
 520.49708,
 487.05792,
 482.934,
 580.15013,
 678.64799,
 769.09804,
 569.22436,
 509.42898,
 542.93203,
 601.17292,
 520.75005,
 573.60697,
 861.65309,
 1233.75106,
 1366.45699,
 702.04496,
 609.74193,
 603.52421,
 577.05188,
 551.55492,
 870.78691,
 947.44325,
 772.16005,
 527.21477,
 509.70602,
 839.42294,
 716.65621,
 570.70589]

<h4>Query 3</h4>

In [78]:
# step 1
large_mongo3 = [{'$match' : {'discipline': 'maths', 'materialType' : 'lecture slides', 'email': {'$regex': '@gmail.com'}}}, {'$group' : {'_id': '_id', 'IDcount' : {'$count': {}}}}]

before = time.time()
large_query3 = largeColl.aggregate(large_mongo3)
after = time.time()

for result in large_query3:
    print(result['IDcount'])

2628


In [79]:
# step 2
msec_duration = (after - before) * 1000
largeDict['query3'].append(round(msec_duration, 5))

In [80]:
# step 3
for i in range(0, 30):
    before = time.time()
    largeColl.aggregate(large_mongo3)
    after = time.time()
    msec_duration = (after - before) * 1000
    largeDict['query3'].append(round(msec_duration, 5))

In [81]:
largeDict['query3']

[514.96506,
 455.28412,
 434.80396,
 513.2432,
 513.92293,
 493.21294,
 463.3081,
 511.21688,
 559.30424,
 463.18507,
 468.4422,
 822.80087,
 471.12799,
 518.98599,
 653.10407,
 552.97112,
 497.51973,
 471.8101,
 447.64519,
 449.89491,
 466.4588,
 478.77407,
 1137.46595,
 680.88794,
 1094.38992,
 773.05913,
 872.89381,
 648.58174,
 538.34081,
 464.5412,
 871.85216]

<h4>Query 4</h4>

In [82]:
# step 1
large_mongo4 = [{'$match' : {'discipline': 'psychology', 'country' : {'$regex': 'orea'}, 'dateOfBirth': {'$regex': '^1'}}}, {'$project' : {'_id': 0, 'firstName': 1, 'lastName': 1, 'country': 1}}, {'$group' : {'_id': {'name': '$firstName', 'surname': '$lastName', 'country': '$country'}}}, {'$sort' : {'lastName' : 1}}]

before = time.time()
large_query4 = largeColl.aggregate(large_mongo4)
after = time.time()

for result in large_query4:
    print(result['_id']['name'], result['_id']['surname'], 'from', result['_id']['country'])

Cathrine Lie from South Korea
Ninthe Horrocks from Noord-Korea
Miguel Real from República de Corea
Lynda Reynolds from Korea
Tere Castells from República Popular Democrática de Corea
Raghav Sura from North Korea


In [83]:
# step 2
msec_duration = (after - before) * 1000
largeDict['query4'].append(round(msec_duration, 5))

In [84]:
# step 3
for i in range(0, 30):
    before = time.time()
    largeColl.aggregate(large_mongo4)
    after = time.time()
    msec_duration = (after - before) * 1000
    largeDict['query4'].append(round(msec_duration, 5))

In [85]:
largeDict['query4']

[502.33507,
 500.86713,
 468.85824,
 497.12586,
 458.06694,
 478.90592,
 495.76187,
 486.49025,
 838.50908,
 563.83181,
 518.32008,
 638.29684,
 728.46889,
 752.55203,
 483.75583,
 590.11388,
 494.96889,
 455.81007,
 697.43729,
 658.54907,
 799.87884,
 512.34794,
 1127.29311,
 561.58686,
 581.60114,
 504.50587,
 490.91005,
 487.00476,
 842.80491,
 1154.19316,
 632.16805]

In [103]:
largeDataset = {'query1' : list(), 'query2' : list(), 'query3' : list(), 'query4' : list()}
for key in largeDict:
    largeDataset[key].append(largeDict[key][0])
    mean30 = mean(largeDict[key][1 : 31])
    largeDataset[key].append(round(mean30, 5))
largeDataset

{'query1': [9782.52077, 633.73331],
 'query2': [653.68295, 672.64531],
 'query3': [514.96506, 592.96764],
 'query4': [502.33507, 616.69949]}

<h3>Collection with 1m documents</h3><br>

<h4>Query 1</h4>

In [87]:
# step 1
humongous_mongo1 = [{'$match' : {'courseID' : 192}}, {'$group' : {'_id' : {'name' : '$firstName', 'surname' : '$lastName'}}}, {'$project' : {'firstName' : 1, 'lastName' : 1}}]

before = time.time()
humongous_query1 = humongousColl.aggregate(humongous_mongo1)
after = time.time()

for result in humongous_query1:
    print(result['_id']['name'], result['_id']['surname'])

Kristen Webb
Arthur Laroche
Eduardo Rezende
David Miranda
Custodia Hidalgo
Nedas Naujokas
Sarah Lara
Narciso Ferrán
Patrícia Leite
Vigilija Gaižauskas
Cathrine Lie
Yuvaan Dara
Melania Savorgnan
Brian Thompson
Collin Heerkens
Mamen Teruel
Ana Narušis
Shaan Raju
Ledün Soylu
Urvi Dani
Giuseppina Scarfoglio
Miguel Real
Finn Karlsen
Torsten Schulz
Özkutlu Gül
Dorita Abella
Christl Henschel
Joris Kavaliauskas
Nath Nicolas
Karl Christensen
Débora Vaz
Ingeborg Amundsen
Casandra Arenas
Liliana Flaiano
Émile Nicolas


In [88]:
# step 2
msec_duration = (after - before) * 1000
humongousDict['query1'].append(round(msec_duration, 5))

In [89]:
# step 3
for i in range(0, 30):
    before = time.time()
    humongousColl.aggregate(humongous_mongo1)
    after = time.time()
    msec_duration = (after - before) * 1000
    humongousDict['query1'].append(round(msec_duration, 5))

In [90]:
humongousDict['query1']

[5834.81503,
 832.49187,
 1290.58981,
 671.86213,
 669.47007,
 576.32494,
 1523.43106,
 1050.98271,
 851.20416,
 558.98714,
 540.64989,
 544.18421,
 526.27993,
 534.65605,
 524.49012,
 548.47217,
 517.32683,
 666.24522,
 653.86796,
 534.01804,
 618.32309,
 574.23377,
 962.80599,
 755.01704,
 776.46089,
 828.9721,
 651.44086,
 712.49723,
 919.90924,
 1001.40595,
 515.172]

<h4>Query 2</h4>

In [91]:
# step 1
humongous_mongo2 = [{'$match' : {'discipline' : 'statistics', 'courseYear' : 2022}}, {'$group' : {'_id' : {'ID' : '$courseID', 'course' : '$courseName'}}}, {'$project' : {'_id.course' : 1}}]

before = time.time()
humongous_query2 = humongousColl.aggregate(humongous_mongo2)
after = time.time()

for result in humongous_query2:
    print(result['_id']['course'])

Bayesian Statistics: From Concept to Data Analysis
Understanding Clinical Research: Behind the Statistics
Python and Statistics for Financial Analysis
Exploratory Data Analysis
Introduction to Statistics
Foundations: Data, Data, Everywhere
Introduction to Probability and Data with R
Basic Statistics
Econometrics: Methods and Applications


In [92]:
# step 2
msec_duration = (after - before) * 1000
humongousDict['query2'].append(round(msec_duration, 5))

In [93]:
# step 3
for i in range(0, 30):
    before = time.time()
    humongousColl.aggregate(humongous_mongo2)
    after = time.time()
    msec_duration = (after - before) * 1000
    humongousDict['query2'].append(round(msec_duration, 5))

In [94]:
humongousDict['query2']

[747.70832,
 730.35312,
 783.49686,
 782.30906,
 808.21609,
 667.05799,
 736.866,
 695.90092,
 929.22592,
 1138.3009,
 1308.84886,
 1411.04293,
 1394.315,
 1130.91207,
 1033.72502,
 777.04096,
 798.99406,
 753.29089,
 705.51419,
 781.56304,
 743.41512,
 1844.29479,
 1168.26105,
 1698.89593,
 781.16012,
 738.36088,
 1406.00181,
 1383.83484,
 746.26279,
 795.04371,
 776.18217]

<h4>Query 3</h4>

In [95]:
# step 1
humongous_mongo3 = [{'$match' : {'discipline': 'maths', 'materialType' : 'lecture slides', 'email': {'$regex': '@gmail.com'}}}, {'$group' : {'_id': '_id', 'IDcount' : {'$count': {}}}}]

before = time.time()
humongous_query3 = humongousColl.aggregate(humongous_mongo3)
after = time.time()

for result in humongous_query3:
    print(result['IDcount'])

3498


In [96]:
# step 2
msec_duration = (after - before) * 1000
humongousDict['query3'].append(round(msec_duration, 5))

In [97]:
# step 3
for i in range(0, 30):
    before = time.time()
    humongousColl.aggregate(humongous_mongo3)
    after = time.time()
    msec_duration = (after - before) * 1000
    humongousDict['query3'].append(round(msec_duration, 5))

In [98]:
humongousDict['query3']

[826.83492,
 829.48995,
 601.92585,
 894.26589,
 840.446,
 1057.68299,
 1011.10816,
 1086.95388,
 833.745,
 1551.10598,
 1147.73202,
 571.22707,
 949.24808,
 668.14017,
 743.96801,
 710.4938,
 576.84898,
 1185.98175,
 726.10712,
 662.17923,
 678.99013,
 687.87193,
 692.00325,
 626.09506,
 769.62399,
 880.98979,
 698.39001,
 622.45822,
 656.358,
 610.86297,
 618.258]

<h4>Query 4</h4>

In [99]:
# step 1
humongous_mongo4 = [{'$match' : {'discipline': 'psychology', 'country' : {'$regex': 'orea'}, 'dateOfBirth': {'$regex': '^1'}}}, {'$project' : {'_id': 0, 'firstName': 1, 'lastName': 1, 'country': 1}}, {'$group' : {'_id': {'name': '$firstName', 'surname': '$lastName', 'country': '$country'}}}, {'$sort' : {'lastName' : 1}}]

before = time.time()
humongous_query4 = humongousColl.aggregate(humongous_mongo4)
after = time.time()

for result in humongous_query4:
    print(result['_id']['name'], result['_id']['surname'], 'from', result['_id']['country'])

Raghav Sura from North Korea
Tere Castells from República Popular Democrática de Corea
Cathrine Lie from South Korea
Leila Gailys from Korea
Miguel Real from República de Corea
Ninthe Horrocks from Noord-Korea
Debra Shaw from Korea
Lynda Reynolds from Korea


In [100]:
# step 2
msec_duration = (after - before) * 1000
humongousDict['query4'].append(round(msec_duration, 5))

In [101]:
# step 3
for i in range(0, 30):
    before = time.time()
    humongousColl.aggregate(humongous_mongo4)
    after = time.time()
    msec_duration = (after - before) * 1000
    humongousDict['query4'].append(round(msec_duration, 5))

In [102]:
humongousDict['query4']

[737.1347,
 670.23516,
 629.88281,
 886.08575,
 711.47394,
 661.36432,
 637.15196,
 814.68081,
 1278.615,
 1231.72092,
 1123.15321,
 1182.34205,
 827.6279,
 587.60381,
 1179.26812,
 853.82009,
 1024.99199,
 723.50693,
 624.17412,
 651.72505,
 1130.81121,
 791.07404,
 755.97191,
 741.24217,
 722.92495,
 658.13208,
 750.82302,
 857.6591,
 702.60668,
 721.60196,
 771.14892]

In [106]:
humongousDataset = {'query1' : list(), 'query2' : list(), 'query3' : list(), 'query4' : list()}
for key in humongousDict:
    humongousDataset[key].append(humongousDict[key][0])
    mean30 = mean(humongousDict[key][1 : 31])
    humongousDataset[key].append(round(mean30, 5))
humongousDataset

{'query1': [5834.81503, 731.05908],
 'query2': [747.70832, 981.6229],
 'query3': [826.83492, 806.35171],
 'query4': [737.1347, 830.114]}

In [107]:
with open('mongo_tests.csv', 'w', newline = '') as mongo_tests:
    writer = csv.writer(mongo_tests, delimiter = ',')
    keys = smallDataset.keys()
    limit = len(smallDataset['query1'])
    
    writer.writerow(keys)
    writer.writerow('s') # s stands for small dataset
    for i in range(0, limit):
        writer.writerow(smallDataset[k][i] for k in keys)
    writer.writerow('m')  # m stands for medium dataset
    for i in range(0, limit):
        writer.writerow(mediumDataset[k][i] for k in keys)
    writer.writerow('l') # l stands for large dataset
    for i in range(0, limit):
        writer.writerow(largeDataset[k][i] for k in keys)
    writer.writerow('h') # h stands for humongous dataset
    for i in range(0, limit):
        writer.writerow(humongousDataset[k][i] for k in keys)